In [1]:
import torch
import adapters

from transformers import AutoModel, AutoModelForCausalLM, AutoConfig
from adapters import LlamaAdapterModel
from transformers import AutoTokenizer
from adapters.models.llama.adapter_model import LlamaAdapterModel

/home/public/vdeshpan/miniconda3/envs/peft_comparison_v2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda")

In [3]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")
#tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-1b")

#
config = AutoConfig.from_pretrained("meta-llama/Llama-2-7b-hf")
#config.load_in_8bit = True

#
model = LlamaAdapterModel.from_pretrained("meta-llama/Llama-2-7b-hf", load_in_4bit=True)
model.add_causal_lm_head("lm_head")

#
model_ref = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf", load_in_4bit=True)


Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.30s/it]


In [4]:
model

LlamaAdapterModel(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayerWithAdapters(
        (self_attn): LlamaAttentionWithAdapters(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
          (prefix_tuning): PrefixTuningShim(
            (prefix_gates): ModuleDict()
            (pool): PrefixTuningPool(
              (prefix_tunings): ModuleDict()
            )
          )
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Line

In [5]:
model.heads.lm_head[0].weight = model_ref.lm_head.weight
del model_ref

In [6]:
model_ref.lm_head.weight

NameError: name 'model_ref' is not defined

In [7]:
model.eval()
for name, par in model.named_parameters():
    par.requires_grad = False

#model.add_adapter("adapter", config="lora", set_active=True)
#model.to(device)

In [8]:
model

LlamaAdapterModel(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayerWithAdapters(
        (self_attn): LlamaAttentionWithAdapters(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
          (prefix_tuning): PrefixTuningShim(
            (prefix_gates): ModuleDict()
            (pool): PrefixTuningPool(
              (prefix_tunings): ModuleDict()
            )
          )
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Line

In [9]:
model.model.layers[0].attention_adapters

AdapterLayer(
  (adapters): ModuleDict()
  (adapter_fusion_layer): ModuleDict()
)

In [10]:
adapters.init(model)

LlamaAdapterModel(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayerWithAdapters(
        (self_attn): LlamaAttentionWithAdapters(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
          (prefix_tuning): PrefixTuningShim(
            (prefix_gates): ModuleDict()
            (pool): PrefixTuningPool(
              (prefix_tunings): ModuleDict()
            )
          )
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Line

In [11]:
model.to(device)

You shouldn't move a model when it is dispatched on multiple devices.


ValueError: `.to` is not supported for `4-bit` or `8-bit` bitsandbytes models. Please use the model as it is, since the model has already been set to the correct devices and casted to the correct `dtype`.

In [12]:
toked = tokenizer.encode_plus(
    "Sentence: I feel good. Sentiment of the previous sentence: Positive. Sentence: I feel bad. Sentiment of the previous sentence: ", return_tensors="pt")
input_ids = toked["input_ids"]
attn_mask = toked["attention_mask"]
input_ids = input_ids.to(device)
attn_mask = attn_mask.to(device)
print(input_ids.shape)
print(input_ids.device)

torch.Size([1, 33])
cuda:0


In [13]:
out_ = model.generate(
    inputs=input_ids,
    max_length=10,
    min_length=2,
    num_beams=5
)

/home/public/vdeshpan/miniconda3/envs/peft_comparison_v2/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:224: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


In [14]:
out_

tensor([[    1, 28048,   663, 29901,   306,  4459,  1781, 29889, 28048,  2073,
           310,   278,  3517, 10541, 29901, 10321,  3321, 29889, 28048,   663,
         29901,   306,  4459,  4319, 29889, 28048,  2073,   310,   278,  3517,
         10541, 29901, 29871, 30191, 29940,   387,  1230, 29889, 28048,   663,
         29901,   306,  4459, 14610, 29889, 28048,  2073,   310,   278,  3517,
         10541, 29901, 29871, 30191, 29903,   328, 29889, 28048,   663, 29901,
           306,  4459,  9796, 29889, 28048,  2073,   310,   278,  3517, 10541,
         29901, 29871, 30191, 29950, 14862, 29889, 28048,   663, 29901,   306,
          4459,   316, 13120, 29889, 28048,  2073,   310,   278,  3517, 10541,
         29901, 29871, 30191,  2772, 13120, 29889, 28048,   663, 29901,   306]],
       device='cuda:0')

In [15]:
tokenizer.batch_decode(out_)

['<s> Sentence: I feel good. Sentiment of the previous sentence: Positive. Sentence: I feel bad. Sentiment of the previous sentence: \xadNegative. Sentence: I feel sad. Sentiment of the previous sentence: \xadSad. Sentence: I feel happy. Sentiment of the previous sentence: \xadHappy. Sentence: I feel depressed. Sentiment of the previous sentence: \xadDepressed. Sentence: I']

In [ ]:
model.add_adapter("adapter", "pfeiffer", set_active=True)
model.train()
model.train_adapter("adapter")

In [ ]:
model.model.layers[0].output_adapters.adapters.adapter.adapter_down[0].weight.device

In [ ]:
model.to(device)

In [ ]:
for name, module in model.named_modules():
    if "adapter" in name:
        module.to(device)

In [ ]:
model.model.layers[0].output_adapters.adapters.adapter.adapter_down[0].weight.device

In [ ]:
model(input_ids=input_ids)